In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import torch
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm_notebook

import fannypack
from lib import panda_datasets, panda_baseline_models, panda_baseline_training


In [10]:
# Experiment configuration
experiment_name = "lstm_test3"
dataset_args = {
    'use_proprioception': True,
    'use_haptics': True,
    'use_vision': True,
    'vision_interval': 2,
}

In [11]:
print("Creating model...")
model = panda_baseline_models.PandaLSTMModel(batch_size=16)
buddy = fannypack.utils.Buddy(experiment_name, model)

Creating model...
[buddy-lstm_test3] Using device: cuda
[buddy-lstm_test3] No checkpoint found


In [12]:
print("Creating dataset...")
dataset = panda_datasets.PandaParticleFilterDataset(
    'data/gentle_push_10.hdf5',
    subsequence_length=16,
    **dataset_args)

Creating dataset...
[[ 0.43320465 -0.06085153]
 [ 0.43342954 -0.06110969]
 [ 0.43332869 -0.0610823 ]
 [ 0.43344337 -0.06091748]
 [ 0.43343791 -0.06095281]
 [ 0.43329573 -0.06094225]
 [ 0.43338215 -0.06090631]
 [ 0.43331465 -0.06094184]
 [ 0.4333204  -0.06095319]
 [ 0.43339431 -0.06094772]
 [ 0.43334761 -0.06092997]
 [ 0.43333849 -0.06098099]
 [ 0.43339148 -0.06096591]
 [ 0.43334946 -0.06092997]
 [ 0.433339   -0.06098142]
 [ 0.4333916  -0.06096607]
 [ 0.43334982 -0.06092998]
 [ 0.43333933 -0.06098147]
 [ 0.43339187 -0.06096612]
 [ 0.43335009 -0.06093002]
 [ 0.4333396  -0.06098151]
 [ 0.43339217 -0.06096617]
 [ 0.43335038 -0.06093006]
 [ 0.43333989 -0.06098155]
 [ 0.43339244 -0.06096621]
 [ 0.43335068 -0.0609301 ]
 [ 0.43334019 -0.06098159]
 [ 0.43339273 -0.06096625]
 [ 0.43335095 -0.06093014]
 [ 0.43334046 -0.06098163]
 [ 0.43339303 -0.06096629]
 [ 0.43335125 -0.06093019]
 [ 0.43334076 -0.06098167]
 [ 0.4333933  -0.06096633]
 [ 0.43335155 -0.06093023]
 [ 0.43334106 -0.06098171]
 [ 0.433

In [ ]:
buddy.set_learning_rate(1e-4)
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=16, shuffle=True, num_workers=2, drop_last=True)
for _ in tqdm_notebook(range(1000)):
    for batch_idx, batch in enumerate(tqdm_notebook(dataloader)):
        _, states, observations, _ = fannypack.utils.to_device(batch, buddy._device)

        model.reset_hidden_states(states[:, 0, :])
        predicted_states = model(observations)
        loss = F.mse_loss(predicted_states, states)
        buddy.minimize(loss, checkpoint_interval=500)
        buddy.log("loss", loss)

buddy.save_checkpoint()

[buddy-lstm_test3] Saved checkpoint to path: checkpoints/lstm_test3-0000000000001000.ckpt



[buddy-lstm_test3] Saved checkpoint to path: checkpoints/lstm_test3-0000000000001500.ckpt



[buddy-lstm_test3] Saved checkpoint to path: checkpoints/lstm_test3-0000000000002000.ckpt



[buddy-lstm_test3] Saved checkpoint to path: checkpoints/lstm_test3-0000000000002500.ckpt



[buddy-lstm_test3] Saved checkpoint to path: checkpoints/lstm_test3-0000000000003000.ckpt



...

[buddy-lstm_test3] Saved checkpoint to path: checkpoints/lstm_test3-0000000000006000.ckpt



[buddy-lstm_test3] Saved checkpoint to path: checkpoints/lstm_test3-0000000000006500.ckpt



[buddy-lstm_test3] Saved checkpoint to path: checkpoints/lstm_test3-0000000000007000.ckpt



[buddy-lstm_test3] Saved checkpoint to path: checkpoints/lstm_test3-0000000000007500.ckpt



[buddy-lstm_test3] Saved checkpoint to path: checkpoints/lstm_test3-0000000000008000.ckpt



In [ ]:
eval_trajectories = panda_datasets.load_trajectories(("data/pushset_small.hdf5",10), **dataset_args)

pred, actual = panda_baseline_training.rollout_lstm(model, eval_trajectories)
panda_baseline_training.eval_rollout(pred, actual, plot=True)

In [ ]:
buddy.save_checkpoint("stable")

In [ ]:
buddy.save_checkpoint()